In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.imgnet_utils import denormalize

from src.data_loader import _create_dataLoader

from src.Dataset import KaggleSafeDriverDataset

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet, MyVGG)
from src.extractor_utils import (predict, getPrediction,features_saving,
                                 features_loading, torch_summarize)

from utils.utils import (create_submission ,metrics2csv, save_results)

# Checking for resources avaliables

In [2]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G        194G        5.0G         10G         20G         14G
Swap:            0B          0B          0B
Sun Nov 26 23:40:43 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   60C    P0   134W / 149W |   7316MiB / 11439MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0

In [3]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

4 GPU's available:

cpu_count: 24


In [4]:
use_gpu = True
use_DataParalel= False 
use_CPU= False       

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))
    else:
        print('Using only one GPU') 
if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using only one GPU



# Path to features

In [5]:
path2features= "./features/" 

# Choosing Model

In [6]:
use_resnet = True
use_inception = False
use_denseNet = False

if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
    model = MyResNet()
elif use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet()
    
elif use_VGG:
    print('Using VGG model')
    model_name = "VGG"    
    model = MyVGG()  

Using ResNet model


In [7]:
if use_gpu:
    if use_DataParalel:
        print("Using all GPU's ")
        model.mrnc = torch.nn.DataParallel(model.mrnc) #device_ids=[1,3]
        model.mrnc = model.mrnc.cuda()
        model.mrnd = torch.nn.DataParallel(model.mrnd) #device_ids=[1,3]
        model.mrnd = model.mrnd.cuda()
    else:
        print('Using GPU')# {}'.format(device_id))
        model.cuda()
else:
    print("Using CPU's")

Using GPU


# Loading features

In [8]:
model_name

'ResNet'

In [9]:
load_feat= features_loading(path2features,model_name)

Loaded features with shapes: 


train:
pred torch.Size([17940, 6400]), true torch.Size([17940])

valid:
pred torch.Size([4484, 6400]), true torch.Size([4484])

test:
pred torch.Size([79726, 6400]), true torch.Size([79726])


In [10]:
load_feat['train'][0].shape,load_feat['train'][1].shape

(torch.Size([17940, 6400]), torch.Size([17940]))

In [11]:
conv_dset ={
'train': TensorDataset(load_feat['train'][0], load_feat['train'][1]),
'valid': TensorDataset(load_feat['valid'][0], load_feat['valid'][1]),
'test': TensorDataset(load_feat['test'][0], load_feat['test'][1])
}

# Creating Dataloaders

In [12]:
batch_size= 32

dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)

In [13]:
dset_convnet_sizes = {x: len(dset_loaders_convnet[x]) for x in ['train','valid', 'test']} 
dset_convnet_sizes

{'test': 2492, 'train': 561, 'valid': 141}

# Trainning Model

In [ ]:
path2saveModel = './models/'+model_name+'DistractDriver' 
savebest = ptt.ModelCheckpoint(path2saveModel,reset=True, verbose=1)

In [15]:
loss_fn = nn.CrossEntropyLoss()
num_epochs = 70

optimizer =  optim.Adam(model.mrnd.parameters(), lr=1e-3,weight_decay=0)
scheduler = StepLR(optimizer, step_size=5, gamma=0.55)

params = {'model' : model.mrnd, 
    'criterion': loss_fn,  
    'optimizer': optimizer, 
    'callbacks': [savebest, ptt.AccuracyMetric(), ptt.PrintCallback()] #ptt.PlotCallback(),
}

In [16]:
trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,**params)

In [17]:
trainer.fit_loader(num_epochs, dset_loaders_convnet['train'], dset_loaders_convnet['valid'])

Start training for 70 epochs
  1:  36.3s   T: 2.23244 0.18779   V: 1.32959 0.47569 best
  2:  14.6s   T: 0.98976 0.62653   V: 0.45969 0.86062 best
  3:  14.6s   T: 0.49378 0.82480   V: 0.30764 0.90299 best
  4:  14.7s   T: 0.34106 0.88590   V: 0.25421 0.90745 best
  5:  14.7s   T: 0.29950 0.89944   V: 0.13302 0.95852 best
  6:  14.7s   T: 0.26380 0.91349   V: 0.13454 0.95919 
  7:  14.7s   T: 0.21400 0.92938   V: 0.13481 0.95763 
  8:  14.8s   T: 0.23064 0.92363   V: 0.26040 0.92730 
  9:  14.7s   T: 0.23723 0.92090   V: 0.12595 0.96566 best
 10:  14.5s   T: 0.28122 0.90139   V: 0.10935 0.96922 best
 11:  14.5s   T: 0.21046 0.92988   V: 0.09464 0.97502 best
 12:  14.4s   T: 0.24003 0.91957   V: 0.06506 0.98417 best
 13:  14.7s   T: 0.25684 0.91382   V: 0.12538 0.96120 
 14:  14.3s   T: 0.21879 0.92726   V: 0.04421 0.98773 best
 15:  14.2s   T: 0.18784 0.93835   V: 0.12535 0.95874 
 16:  13.9s   T: 0.17018 0.94387   V: 0.07405 0.98060 
 17:  14.1s   T: 0.17611 0.93974   V: 0.06733 0.980

In [43]:

print(torch_summarize(model))

MyResNet (
  (mrnc): MyResNetConv (
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU (inplace)
    (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (layer1): Sequential (
      (0): BasicBlock (
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU (inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      )
      (1): BasicBlock (
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU (inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride

# Loading best epoch

In [21]:
trainer.load_state(path2saveModel)
model.mrnd = trainer.model

In [22]:
metrics2csv(trainer, model_name)

done!


In [23]:
train_eval = trainer.evaluate_loader(dset_loaders_convnet['train'])
valid_eval = trainer.evaluate_loader(dset_loaders_convnet['valid'])

evaluate: 560/560 ok
evaluate: 140/140 ok


In [24]:
train_eval, valid_eval

({'losses': 0.013342254084750772}, {'losses': 0.03572719438495006})

# Dataset without shuffling 

#### Utilized to plot mismatch cases

In [25]:
path2train = "/mnt/home/e209440/data/train"  

In [26]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=300 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}   

In [27]:
batch_size = 32
use_only = 1.0 # Use only is the percentage of the full dataset

In [28]:
dsets = {
    'valid': KaggleSafeDriverDataset(path2train,
                                     transforms=data_transforms['valid'],
                                     use_only=use_only, is_val=True, val_size=0.2),
}

In [29]:

dset_loaders_wshuffle = _create_dataLoader(dsets, batch_size,
                                           pin_memory=False, use_shuffle= False)

# Evaluating resultings 

In [30]:
#we need to use all dloader, because this one has use_shuffle false
result_train = predict(dset_loaders_convnet['train'], model.mrnd, use_gpu=use_gpu)

result_valid = predict(dset_loaders_wshuffle['valid'], model, use_gpu=use_gpu)

result_test = predict(dset_loaders_convnet['test'], model.mrnd, use_gpu=use_gpu)

predict: 560/560 ok
predict: 140/140 ok
predict: 2491/2491 ok


In [31]:
predictions_out = {'train': result_train, 'valid': result_valid, 'test': result_test}

In [32]:
save_results(predictions_out, model_name,use_gpu)

train set result saved!
valid set result saved!
test set result saved!


In [33]:
result_train = getPrediction(result_train)
result_valid = getPrediction(result_valid)
# result_test['pred'] must be an array of probabilities to make the submission

In [34]:
correct_train = (result_train['true'] == result_train['pred']).sum()
correct_valid = (result_valid['true'] == result_valid['pred']).sum()

In [35]:
print('Train: ', correct_train, '/', '17940' )# len(dsets['train'])
print('Valid: ', correct_valid, '/', '4484' ) # len(dsets['valid'])

Train:  17879 / 17940
Valid:  4438 / 4484


# Make submission of the Test set

In [36]:
create_submission(result_test, model_name + '_distracted_driver')

done!
